### Additions in Mistral model compared ot the original transformer:- 

1. Rotary Positional Embeddings (introduced in llama)
2. SwiGLU activation function (introduced in llama)
3. RMS Norm (introduced in llama)
4. Grouped Query attention (introduced in llama)

5. **Sliding window attention (New)**

6. KV Caching :- 
    - **Rolling buffer cache**
    - **Prefilling and chunking**

### Specific to Mixtral 8x7B

#### Mixture of experts 

- The feedforward networks are modified to get the mixtrue of experts output.
- Each token is (token here is the embedding vector after self_attn + normalization) fed into a gate.
- This gate decides which 2 experts to use out of all 8 experts
- Then, the outputs are obtained from the selected experts and final output is the weighted sum of the induvidual output.
- These final outputs are then reshaped into (bsz, seq_len, embed_dim)

Here an expert is a feedforward layer (the one from llama2)

In this way each embedding vector i.e each token is passed through 2 different feedforward networks and combined to get the final output out of the decoder block.

In [1]:
import torch

selected_experts = torch.tensor([[0, 2], [1, 3], [2, 0]])  # Assuming 3 tokens with top 2 experts selected for each
num_experts = 4

# Creating one-hot encoded expert mask
expert_mask = torch.nn.functional.one_hot(selected_experts, num_classes=num_experts).permute(2, 1, 0)

print(expert_mask)

tensor([[[1, 0, 0],
         [0, 0, 1]],

        [[0, 1, 0],
         [0, 0, 0]],

        [[0, 0, 1],
         [1, 0, 0]],

        [[0, 0, 0],
         [0, 1, 0]]])


In [3]:
torch.nn.functional.one_hot(selected_experts, num_classes=num_experts), torch.nn.functional.one_hot(selected_experts, num_classes=num_experts).shape 


(tensor([[[1, 0, 0, 0],
          [0, 0, 1, 0]],
 
         [[0, 1, 0, 0],
          [0, 0, 0, 1]],
 
         [[0, 0, 1, 0],
          [1, 0, 0, 0]]]),
 torch.Size([3, 2, 4]))